# Objective

* Make a baseline model that predict the validation (28 days). 
* This competition has 2 stages, so the main objective is to make a model that can predict the demand for the next 28 days

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

from sklearn import preprocessing, metrics
import gc
import os


In [6]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


# function to read the data and merge it (ignoring some columns, this is a very fst model)


def read_data():
    print('Reading files...')
    calendar = pd.read_csv('inputs/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv('inputs/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv('inputs/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv('inputs/sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission


def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    # get only a sample for fst training
    data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data
        
calendar, sell_prices, sales_train_validation, submission = read_data()
data = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 27500000, merge = True)

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Melted sales train validation has 58327370 rows and 8 columns
Mem. usage decreased to 3226.27 Mb (9.4% reduction)
Our final dataset to train has 31681090 rows and 18 columns


In [8]:
data.tail(50)

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
31681040,FOODS_3_778_WI_3_validation,FOODS_3_778,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.279297
31681041,FOODS_3_779_WI_3_validation,FOODS_3_779,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.480469
31681042,FOODS_3_780_WI_3_validation,FOODS_3_780,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.980469
31681043,FOODS_3_781_WI_3_validation,FOODS_3_781,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.480469
31681044,FOODS_3_782_WI_3_validation,FOODS_3_782,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.500000
31681045,FOODS_3_783_WI_3_validation,FOODS_3_783,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,5.441406
31681046,FOODS_3_784_WI_3_validation,FOODS_3_784,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.480469
31681047,FOODS_3_785_WI_3_validation,FOODS_3_785,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,3.000000
31681048,FOODS_3_786_WI_3_validation,FOODS_3_786,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.980469
31681049,FOODS_3_787_WI_3_validation,FOODS_3_787,FOODS_3,FOODS,WI_3,WI,0,test1,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.480469


* We have the data to build our first model, let's build a baseline and predict the validation data (in our case is test1)

In [9]:
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    
    return data

def simple_fe(data):
    
    # rolling demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())
    
    
    # price features
    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    
    return data

def run_lgb(data):
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-03-27']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    y_val = x_val['demand']
    test = data[(data['date'] > '2016-04-24')]
    del data
    gc.collect()

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 236,
        'learning_rate': 0.1,
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75}

    train_set = lgb.Dataset(x_train[features], y_train)
    val_set = lgb.Dataset(x_val[features], y_val)
    
    del x_train, y_train

    model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [train_set, val_set], verbose_eval = 100)
    val_pred = model.predict(x_val[features])
    val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
    print(f'Our val rmse score is {val_score}')
    y_pred = model.predict(test[features])
    test['demand'] = y_pred
    return test

def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submissions/submission.csv', index = False)
    

# define list of features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30']


def transform_train_and_eval(data):
    data = transform(data)
    data = simple_fe(data)
    # reduce memory for new features so we can train
    data = reduce_mem_usage(data)
    test = run_lgb(data)
    predict(test, submission)
    


In [10]:
data = transform(data)

In [12]:
data = simple_fe(data)


In [14]:
data.tail(50)

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_t28,lag_t29,lag_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,month,week,day,dayofweek
31681040,FOODS_3_778_WI_3_validation,1387,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.279297,1.0,1.0,1.0,1.000000,8.164966e-01,0.400000,0.244444,0.205556,0.723974,1.542240,0.877440,0.0,0.000000,0.000000e+00,5.960464e-08,2016,5,20,22,6
31681041,FOODS_3_779_WI_3_validation,1388,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.480469,0.0,2.0,0.0,0.428571,7.867958e-01,0.366667,0.222222,0.222222,0.850287,3.152666,11.481625,0.0,0.000000,0.000000e+00,0.000000e+00,2016,5,20,22,6
31681042,FOODS_3_780_WI_3_validation,1389,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.980469,1.0,0.0,2.0,1.285714,7.559289e-01,1.466667,1.188889,1.127778,1.407696,1.145627,1.005601,0.0,0.000000,0.000000e+00,0.000000e+00,2016,5,20,22,6
31681043,FOODS_3_781_WI_3_validation,1390,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.480469,0.0,0.0,0.0,0.571429,9.759001e-01,0.566667,0.622222,0.572222,0.971431,1.975527,4.192329,0.0,0.000000,0.000000e+00,0.000000e+00,2016,5,20,22,6
31681044,FOODS_3_782_WI_3_validation,1391,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.500000,9.0,6.0,5.0,4.428571,2.636737e+00,3.733333,3.211111,3.377778,2.753472,0.807845,0.339966,0.0,0.081116,0.000000e+00,7.727051e-02,2016,5,20,22,6
31681045,FOODS_3_783_WI_3_validation,1392,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,5.441406,0.0,0.0,0.0,0.000000,1.515358e-07,0.633333,1.611111,1.133333,1.159171,2.356439,6.265667,0.0,0.007126,5.960464e-08,0.000000e+00,2016,5,20,22,6
31681046,FOODS_3_784_WI_3_validation,1393,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,1.480469,0.0,0.0,0.0,0.571429,7.867958e-01,1.033333,0.833333,0.422222,1.542129,2.418798,7.107114,0.0,0.000000,0.000000e+00,0.000000e+00,2016,5,20,22,6
31681047,FOODS_3_785_WI_3_validation,1394,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,3.000000,16.0,28.0,10.0,14.142857,1.246901e+01,3.300000,1.100000,6.533333,8.317617,2.637058,6.327701,0.0,0.000000,0.000000e+00,0.000000e+00,2016,5,20,22,6
31681048,FOODS_3_786_WI_3_validation,1395,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,1.980469,1.0,3.0,4.0,1.857143,1.463850e+00,1.433333,1.444444,1.338889,1.165106,0.312606,-0.842109,0.0,0.000000,0.000000e+00,0.000000e+00,2016,5,20,22,6
31681049,FOODS_3_787_WI_3_validation,1396,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.480469,0.0,1.0,0.0,0.428571,5.345225e-01,0.766667,0.900000,0.755556,0.858360,1.188121,1.266307,0.0,0.000000,5.960464e-08,0.000000e+00,2016,5,20,22,6


In [ ]:
transform_train_and_eval(data)